# Person information retrieval system ✍️

---
Maite Heredia & Mikel Idoyaga



## Imports and installations

In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [2]:
!pip install Whoosh
!pip install transformers
!pip3 install wikipedia-api
!pip install datasets
!pip install seqeval
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 9.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [34]:
import locale
#locale.getpreferredencoding = lambda: "UTF-8"
import codecs # base classes for standard Python codecs, like text encodings (UTF-8,...)
from IPython.core.display import display, HTML # object displaying in different formats
from whoosh.index import * # whoosh: full-text indexing and searching
from whoosh.fields import *
from whoosh import qparser
import glob
import random
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM
from transformers import pipeline
import pandas as pd
import wikipediaapi
import re
from datasets import load_dataset
import evaluate
import ast
from datasets import load_dataset
wiki_wiki = wikipediaapi.Wikipedia('en')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def tokenInd (target,text):
  textlist = text.split()
  sum = 0
  for i in range(len(textlist)):
    sum += len(textlist[i])+1

    if sum> target:
      ind = i
      break

  return ind

## Definition, initialization and evaluation of systems

### NER

In [6]:
# esta va bien
class NERtagger(object):
  def __init__(self):
    self.tokenizer = AutoTokenizer.from_pretrained("Davlan/distilbert-base-multilingual-cased-ner-hrl")
    self.model = AutoModelForTokenClassification.from_pretrained("Davlan/distilbert-base-multilingual-cased-ner-hrl")
    self.nlp = pipeline("ner", model=self.model, tokenizer=self.tokenizer)
  
  def tag(self,text):
    # We tag the given text
    ner_results = self.nlp(text)
    # We only want to return the tagged names (PER), and reconstruct multiword entities as well as join sub-word tokens.
    entities = {}
    name = ""
    first = True
    for e in ner_results:
      if e["entity"] == "B-PER" and not first:
        if name in entities:
          entities[name].append((tokenInd(start,text) ,tokenInd(end,text)))
        else:
          entities[name] = [(tokenInd(start,text) ,tokenInd(end,text))]
        name = ""
        first = True

      if e["entity"] == "B-PER" and first:
        name = e["word"]
        start = e["start"]
        end = e["end"]
        first = False

      if e["entity"] == "I-PER":
        end = e["end"]
        if e["word"][0] == "#":
          name += e["word"].replace("##","")
        else:
          if e["word"] == ".":
            name += e["word"]
          elif e["word"] == "-":
            name += e["word"]
          elif name[-1] == "-":
            name += e["word"]
          else:
            name += " "+e["word"]   
    
    if name != "":
      if name in entities:
        entities[name].append((tokenInd(start,text) ,tokenInd(end,text)))
      else:
        entities[name] = [(tokenInd(start,text) ,tokenInd(end,text))]
    
    return entities

  def evaluate(self,dataset="conll2003"):
    dataset = load_dataset(dataset)
    metric = evaluate.load("seqeval")
    predictions = []
    for textlist in dataset["test"]["tokens"]:
      aux = ["O"] * len(textlist)
      text=(" ").join(textlist)
      result = self.nlp(text)
      for e in result:
        aux[tokenInd(e["start"],text)] =  e["entity"]
      predictions.append(aux)
    ner_feature = dataset["train"].features["ner_tags"]
    label_names = ner_feature.feature.names
    labels = dataset["test"]["ner_tags"]
    labels = [[label_names[i] for i in label] for label in labels]
    return metric.compute(predictions=predictions, references=labels)

In [7]:
NERmodel = NERtagger()

#### Evaluation

In [8]:
evaluation = NERmodel.evaluate()
evaluation["PER"]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'precision': 0.9530369442705072,
 'recall': 0.9412492269635127,
 'f1': 0.9471064094586186,
 'number': 1617}

### ENTITY DISAMBIGUATION

In [9]:
class ED(object):
  def __init__(self):
    self.tokenizer = AutoTokenizer.from_pretrained("facebook/mgenre-wiki")
    self.model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mgenre-wiki").eval()
  
  def disambiguate(self, sentence):
      outputs = self.model.generate(
          **self.tokenizer(sentence, return_tensors="pt"),
          num_beams=5,
          num_return_sequences=5,
          # OPTIONAL: use constrained beam search
          # prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist()),
      )
      return self.tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [10]:
EDmodel = ED()

### QUESTION ANSWERING

In [11]:
class QA(object):
  def __init__(self):
        self.model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
        self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

  def answer_question(self,query,passage):
        '''
        Takes a `question` string and an `answer_text` string (which contains the
        answer), and identifies the words within the `answer_text` that are the
        answer. Prints them out.
        '''
        # ======== Tokenize ========
        # Apply the tokenizer to the input text, treating them as a text-pair.
        input_ids = self.tokenizer.encode(query, passage)
        tokens = self.tokenizer.convert_ids_to_tokens(input_ids)

        # ======== Set Segment IDs ========
        # Search the input_ids for the first instance of the `[SEP]` token.
        sep_index = input_ids.index(self.tokenizer.sep_token_id)

        # The number of segment A tokens includes the [SEP] token istelf.
        num_seg_a = sep_index + 1

        # The remainder are segment B.
        num_seg_b = len(input_ids) - num_seg_a

        # Construct the list of 0s and 1s.
        segment_ids = [0]*num_seg_a + [1]*num_seg_b

        # There should be a segment_id for every input token.
        assert len(segment_ids) == len(input_ids)

        # ======== Evaluate ========
        # Run our example through the model.
        outputs = self.model(torch.tensor([input_ids]), # The tokens representing our input text.
                        token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                        return_dict=True) 

        start_scores = outputs.start_logits
        end_scores = outputs.end_logits
        # Find the tokens with the highest `start` and `end` scores.
        #answer_start = torch.argmax(start_scores)
        #answer_end = torch.argmax(end_scores)
        max = float('-inf')
        for i in range(len(start_scores[0][sep_index+1:-1])):
          aux = (start_scores[0][i+sep_index+1]+torch.max(end_scores[0][(i+sep_index+1):-1]))/2
          if aux > max:
            max=aux
            answer_start = i +sep_index+1
            answer_end = i+torch.argmax(end_scores[0][(i+sep_index+1):-1]) +sep_index+1


        # Start with the first token.
        answer = tokens[answer_start]

        # Select the remaining answer tokens and join them with whitespace.
        for i in range(answer_start + 1, answer_end + 1):
            
            # If it's a subword token, then recombine it with the previous token.
            if tokens[i][0:2] == '##':
                answer += tokens[i][2:]
            
            # Otherwise, add a space then the token.
            else:
                answer += ' ' + tokens[i]

        return answer

  def evaluate(self,path,joint = False, num_examples=100):
    df = pd.read_csv(path,sep="\t",dtype={"name":str,"year":str,"occupation":str})
    data = df[:num_examples]
    hits = 0

    for index, row in data.iterrows(): 
      query = "When was " + row["name"] + " born?"
      if joint:
        dis_context = "[START] "+row["name"]+" [END] has worked as " + row["occupation"]
        disambiguation = EDmodel.disambiguate(dis_context)
        for d in disambiguation:
          page = wiki_wiki.page(d[:-6])
          prediction = "?"
          if page.exists():
            full_answer = QAmodel.answer_question(query, page.summary[:512])
            if re.search(r"\d\d\d\d", full_answer):
              prediction = re.search(r"\d\d\d\d", full_answer).group(0)
              if prediction == row["year"]:
                hits += 1
            break
      
      else:
        page = wiki_wiki.page(row["name"].replace(" ","_"))
        if page.exists():
          full_answer = QAmodel.answer_question(query, page.summary[:512])
          if re.search(r"\d\d\d\d", full_answer):
            prediction = re.search(r"\d\d\d\d", full_answer).group(0)
            if prediction == row["year"]:
              hits += 1
      
    accuracy = hits/num_examples
    return accuracy

In [12]:
QAmodel = QA()

Accuracy with disambiguation

In [ ]:
QAmodel.evaluate("/content/drive/MyDrive/FinalProjectApp2/data/dataqa.tsv",True)

0.81

Accuracy without disambiguation

In [ ]:
QAmodel.evaluate("/content/drive/MyDrive/FinalProjectApp2/data/dataqa.tsv",False)

0.78

## Pipeline demo

In [ ]:
def demo(text):
  list_punct = ["...",".",",","!","?"]
  final = text.split()
  result = NERmodel.tag(text)
  for name in result:
    for apparition in result[name]:
      auxiliar =  text.split()
      auxiliar[apparition[0]] = "[START] "+auxiliar[apparition[0]]
      for punct in list_punct:
        auxiliar[apparition[1]] = auxiliar[apparition[1]].replace(punct," [END]"+punct)+" [END]"
        auxiliar[apparition[1]] = auxiliar[apparition[1]].replace(" [END]"+punct+" [END]"," [END]"+punct)
      disambiguation = EDmodel.disambiguate((" ").join(auxiliar))
      query = "When was " + name + " born?"
      for d in disambiguation:
        page = wiki_wiki.page(d[:-6])
        prediction = "?"
        if page.exists():
          full_answer = QAmodel.answer_question(query, page.summary[:512])
          if re.search(r"\d\d\d\d", full_answer):
            prediction = re.search(r"\d\d\d\d", full_answer).group(0)
          break

      final[apparition[1]] =  final[apparition[1]] + " ("+ prediction +")"
      for punct in list_punct:
        final[apparition[1]] =  final[apparition[1]].replace(punct+" ("+prediction+")"," ("+prediction+") "+punct)

  return (" ").join(final)

In [ ]:
sentence = "Dwayne Johnson has been sharing amazing photos from the Gala. She was able to pose with most of the Latinos in attendance, like JLo, Anitta, Maluma, and but what disappointed she didn’t run into Jenna Ortega, Pedro Pascal, or Bad Bunny..."
print(demo(sentence))

Hello, mister Harrison Ford (1942) from Indiana Jones (1981) .


## Evaluation

In [32]:
df = pd.read_csv("/content/drive/MyDrive/FinalProjectApp2/data/FinalTest.tsv",sep="\t")
df["result"][0]

'{"Pedro Sánchez": {"pos": [(2,3)], "year":[1972]},"Biden": {"pos": [(6,6)], "year":[1942]}}'

In [30]:
def finalevaluation(text):
  list_punct = ["...",".",",","!","?"]
  final = text.split()
  result = NERmodel.tag(text)
  finaldict = {}
  for name in result:
    for apparition in result[name]:
      auxiliar =  text.split()
      auxiliar[apparition[0]] = "[START] "+auxiliar[apparition[0]]
      for punct in list_punct:
        auxiliar[apparition[1]] = auxiliar[apparition[1]].replace(punct," [END]"+punct)+" [END]"
        auxiliar[apparition[1]] = auxiliar[apparition[1]].replace(" [END]"+punct+" [END]"," [END]"+punct)
      disambiguation = EDmodel.disambiguate((" ").join(auxiliar))
      query = "When was " + name + " born?"
      for d in disambiguation:
        page = wiki_wiki.page(d[:-6])
        prediction = "?"
        if page.exists():
          full_answer = QAmodel.answer_question(query, page.summary[:512])
          if re.search(r"\d\d+", full_answer):
            prediction = re.search(r"\d\d+", full_answer).group(0)
          break
      if name in finaldict:
        finaldict[name]["pos"].append((apparition[0],apparition[1]))
        if prediction == "?":
          finaldict[name]["year"].append(prediction) 
        else:
          finaldict[name]["year"].append(int(prediction)) 
      else:
        if prediction == "?":
          dicti={"pos": [(apparition[0],apparition[1])], "year":[prediction] }
        else:
          dicti={"pos": [(apparition[0],apparition[1])], "year":[int(prediction)] }
        finaldict[name]=dicti
  return finaldict

In [35]:
correct = 0
for index, row in df.iterrows():
  auxdict = ast.literal_eval(row['result'])
  preddict = finalevaluation(row['text'])
  if auxdict == preddict:
    correct += 1
  else:
    print(auxdict)
    print(preddict)
print("Acuracy:"+ str(correct/len(df)*100))

{'Issa Rae': {'pos': [(1, 2)], 'year': [1985]}, 'Ritu Arya': {'pos': [(7, 8)], 'year': [1988]}, 'Ana Cruz Kayne': {'pos': [(13, 15)], 'year': [1990]}}
{'Issa Rae': {'pos': [(1, 2)], 'year': [1985]}, 'Barbie': {'pos': [(5, 5), (11, 11), (20, 20), (30, 30)], 'year': [1959, 1959, 1980, 1959]}, 'Ritu Arya': {'pos': [(7, 8)], 'year': [1988]}, 'Ana Cruz Kayne': {'pos': [(13, 15)], 'year': [1984]}}
{'Michael Cera': {'pos': [(2, 3)], 'year': [1988]}, 'Emerald Fennell': {'pos': [(5, 6)], 'year': [1985]}, 'Helen Mirren': {'pos': [(21, 22)], 'year': [1945]}}
{'Michael Cera': {'pos': [(2, 3)], 'year': [1988]}, 'Emerald Fennell': {'pos': [(5, 6)], 'year': [1985]}, 'Alan': {'pos': [(10, 10)], 'year': ['?']}, 'Midge': {'pos': [(12, 12)], 'year': [1963]}, 'Ken': {'pos': [(17, 17)], 'year': [1961]}, 'Barbie': {'pos': [(19, 19)], 'year': [1959]}, 'Helen Mirren': {'pos': [(21, 22)], 'year': [1945]}}
{'Michael Jordan': {'pos': [(4, 5), (10, 11)], 'year': [1963, 1986]}}
{'Michael Jordan': {'pos': [(4, 5), 

**Error analysis:**
- It seems like the biggest error is the NER system overpredicting entities, so some film and series names are predicted as PER when they should not be considered.
- There are also some errors with the disambiguation system, but they are minor, and most entities are correctly disambiguated.